In [3]:
import alphaspace2 as al
import mdtraj
import numpy as np
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import fcluster, linkage
from alphaspace2.functions import _binCluster, _group
from alphaspace2.Cluster import _DPocket

In [4]:
import features  ## python module with pseudomolecular features
import glob
from collections import defaultdict 
import matplotlib.pyplot as plt
import pandas as pd

### For this tutorial, We use pocket features to compare the pocket/ligand similarities of 10 HIV-protease inhibitors

In [5]:
protease_list = ['1c70','1hvi','1hvj','1izh','1pro','1siv','2i0a','2psv','2q5k','3lzu']

### Calculating the contact pockets for proteases

In [6]:
protease_data_lig = {}
protease_data_ss = {}
protease_data_prot = {}
for pdb_id in protease_list:
    prot = mdtraj.load('Beta_Cluster_features/protein_' + pdb_id + '.pdb')
    protease_data_prot[pdb_id] = prot
    lig = mdtraj.load('Beta_Cluster_features/ligand_' + pdb_id + '.pdb')
    protease_data_lig[pdb_id] = lig
    ss_prot = al.Snapshot()
    ss_prot.run(prot, lig)
    protease_data_ss[pdb_id] = ss_prot
    

OSError: No such file: Beta_Cluster_features/protein_1c70.pdb

In [ ]:
protease_contact_pockets = defaultdict(dict)
for pdb_id in protease_data_ss:
    for px,pocket in enumerate(protease_data_ss[pdb_id].pockets):
        if pocket.isContact:
            protease_contact_pockets[pdb_id][px] = np.array([b.xyz for b in pocket.betas])
            

### Calculate some beta cluster features 

In [ ]:
protease_contact_pockets = defaultdict(dict)
for pdb_id in protease_data_ss:
    for px,pocket in enumerate(protease_data_ss[pdb_id].pockets):
        if pocket.isContact:
            protease_contact_pockets[pdb_id][px] = np.array([b.xyz for b in pocket.betas])
            
protease_pocket_props_dict = {}
for pdb_id in protease_contact_pockets:
    contact_betas = []
    prot = protease_data_prot[pdb_id]
    for px in protease_contact_pockets[pdb_id]:
        contact_betas.extend(protease_contact_pockets[pdb_id][px])
    contact_betas = np.array(contact_betas)
    beta_temp_dict = {}
    beta_temp_dict['occluded_asa'] = features._get_pharmacophore_fingerprint(prot,contact_betas)
    beta_temp_dict['usr'] = features._Get_USR_alpha_beta(contact_betas)
    beta_temp_dict['normalized_PMI'] = features._calculate_NormalizedRatioPMI(contact_betas)
    beta_temp_dict['span'] = features._calculate_SPAN(contact_betas)
    beta_temp_dict['radius_of_gyration'] = features._calculate_RadiusofGyration(contact_betas)    
    protease_pocket_props_dict[pdb_id] = beta_temp_dict     

### Repeat calculation for ligands

In [ ]:
protease_ligand_props_dict = {}
for pdb_id in protease_data_prot:
    lig = protease_data_lig[pdb_id]
    prot = protease_data_prot[pdb_id]
    lig_coords = lig.xyz[0]*10   ### multiply by 10 to convert nm to Angstrom
    lig_temp_dict = {}
    lig_temp_dict['occluded_asa'] = features._get_pharmacophore_fingerprint(prot,lig_coords)
    lig_temp_dict['usr'] = features._Get_USR_alpha_beta(lig_coords)
    lig_temp_dict['normalized_PMI'] = features._calculate_NormalizedRatioPMI(lig_coords)
    lig_temp_dict['span'] = features._calculate_SPAN(lig_coords)
    lig_temp_dict['radius_of_gyration'] = features._calculate_RadiusofGyration(lig_coords)    
    protease_ligand_props_dict[pdb_id] = lig_temp_dict         

### Save data into a pandas dataframe

In [ ]:
pocket_ligand_data = pd.DataFrame()

In [ ]:
field_subfields = {'occluded_asa':['Total_OASA', 'Positive_OASA', 'Negative_OASA', 'H_bond_Donor_OASA', 'H_bond_Acceptor_OASA', 'H_bond_Doneptor_OASA', 'Aromatic_OASA', 'Hydrophobic_OASA', 'Polar_OASA', 'Null_type_OASA'],
                  'usr':['ctd_1', 'ctd_2', 'ctd_3', 'cst_1', 'cst_2', 'cst_3', 'fct_1', 'fct_2', 'fct_3', 'ftf_1', 'ftf_2', 'ftf_3'],
                  'normalized_PMI':['NPR1', 'NPR2']}

In [ ]:
pdb_id = list(protease_pocket_props_dict.keys())
pocket_ligand_data['PDB_ID'] = pdb_id
for field in field_subfields:
    for sub in field_subfields[field]:
        temp_row = []
        for pdb in pdb_id:
            temp_row.append(protease_pocket_props_dict[pdb][field][sub])
        pocket_ligand_data['protein_'+sub] = temp_row
    for sub in field_subfields[field]:
        temp_row = []
        for pdb in pdb_id:
            temp_row.append(protease_ligand_props_dict[pdb][field][sub])
        pocket_ligand_data['ligand_'+sub] = temp_row

for field in ['span', 'radius_of_gyration']:
    temp_row = []
    for pdb in pdb_id:
        temp_row.append(protease_pocket_props_dict[pdb][field])
    pocket_ligand_data['protein_'+field] = temp_row
    temp_row = []
    for pdb in pdb_id:
        temp_row.append(protease_ligand_props_dict[pdb][field])
    pocket_ligand_data['ligand_'+field] = temp_row
        

### Displaying properties in dataframe

In [ ]:
pocket_ligand_data[['PDB_ID','protein_Total_OASA','ligand_Total_OASA','protein_span','ligand_span','protein_radius_of_gyration','ligand_radius_of_gyration']]

### Beta Clusters allow for the direct comparison of ligands and their binding site pockets. Here we calculate the similarity of the ligands and beta pockets based on their occluded surface area features 

In [ ]:
temp_row = []
for pdb_id in protease_list:
    oasa_pocket = [s for _,s in protease_pocket_props_dict[pdb_id]['occluded_asa'].items()]
    oasa_ligand = [s for _,s in protease_ligand_props_dict[pdb_id]['occluded_asa'].items()]
    similarity = 1 - features._soergel(oasa_pocket,oasa_ligand)
    temp_row.append(similarity)

pocket_ligand_data['OASA_similarity'] = temp_row
pocket_ligand_data[['PDB_ID','OASA_similarity']]

### Likewise, we calculate their similarity based on USR features 

In [ ]:
temp_row = []
for pdb_id in protease_list:
    usr_pocket = [s for _,s in protease_pocket_props_dict[pdb_id]['usr'].items()]
    usr_ligand = [s for _,s in protease_ligand_props_dict[pdb_id]['usr'].items()]
    similarity = 1 - features._soergel(usr_pocket,usr_ligand)
    temp_row.append(similarity)

pocket_ligand_data['USR_similarity'] = temp_row
pocket_ligand_data[['PDB_ID','USR_similarity']]